In [1]:
from whispr import *
import pandas as pd

In [2]:
%%capture

'''
To add:


* update volumes on source plate at end (requires better keeping track of volume used)

* better commenting, use of functions / improving of readability

'''



In [11]:
"""
For user to change according to run
"""


file_name = "C:/Users/rycar/Documents/GitHub/WHISPR/source_plates/220107_blCRISPRa_take2.xlsx"
#CSV file with source plate info. Columns are label, well, concentration, volume
source_plate_df = pd.read_excel(file_name, sheet_name = 'ECHO_source_plate', index_col = 0)

# CSV file with output plate layout: columns labeled 1-12 and rows labeled A-H for 96 well plate
output_plate = pd.read_excel(file_name, sheet_name = "plate_layout", index_col = 0, dtype = str)

#CSV file with each reaction as rows and the volume of each input (columns) added to reach final reaction volume
mixing_table_df = pd.read_excel(file_name, sheet_name = "mixing_table", index_col = 0, dtype = str).fillna(0)

source_plate_type = '384PP_AQ_BP' 

checkInputs(source_plate_df, mixing_table_df,source_plate_type)

#specify rxn_vol (default = 2.5) and total_vol (default = 10) if you'd like to change the volume of each individual replicate or the total reaction volume
vol_table_df = generateVolumeTable(mixing_table_df, source_plate_df)

output_df = writeProtocol(source_plate_type, vol_table_df, source_plate_df, output_plate,source_plate_df)
filename = '220107_blCRISPRa_take2.csv'
output_df.to_csv('protocols/'+filename,index = False)

0
0
5
2
0.5
0
0
10
0.2
0
0
0
0
0
0
0
0
0
0
5
2
0.5
0
0
1
0.2
0
0
0
0
0
0
0
0
0
0
5
2
0.5
0
0
0.1
0.2
0
0
0
0
0
0
0
0
0
0
5
2
0.5
0
0
0.01
0.2
0
0
0
0
0
0
0
0
0
0
5
2
0.5
0
0
0
0.2
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
20
4
0
0
10
1
10000
0
0
0
0
2
0
0
1
0
0
20
4
0
0
10
0
10000
0
0
0
0
2
0
0
0
0
0
20
4
10
1
0
0
10000
0
0
0
0
2
0
0
0
0
0
20
4
10
0
0
1
10000
0
0
0
0
2
0
10
1
0
0
20
4
0
0
0
0
10000
0
0
0
0
2
0
10
0
0
0
20
4
0
1
0
0
10000
0
0
0
5
2
0
0
0
0
0
0
0
0
0
10
1
0
0
0
0
5
2
0
0
0
0
0
0
0
10
1
0
0
0
0
0
0
5
2
0
10
1
0
0
0
0
0
0
0
0
0
0
20
4
0
2
0
0
1
0
0
0
0
0
0
0
0
0
250000
20
4
0
2
0
1
1
0
0
0
0
0
0
0
0
0
250000
20
4
0
2
0
7
1
0
0
0
0
0
0
0
0
0
250000
20
4
0
2
0
13.5
1
0
0
0
0
0
0
0
0
0
250000
20
4
0
2
0
0
1
0
0
0
0
0
0
0
0
0
0
20
4
0
2
0
1
1
0
0
0
0
0
0
0
0
0
0
20
4
0
2
0
7
1
0
0
0
0
0
0
0
0
0
0
20
4
0
2
0
13.5
1
0
0
0
0
0
0
0
0
0
0
Volumes used from each well for this protocol:
{'A1': 20.1, 'C7': 1.35, 'C15': 2.2, 'C17': 1.2, 'B4': 4.5, 'B1': 2.475, 'B2': 0.225, 'D1': 0.125, 'D8': 3

In [3]:
mixing_table_df.columns
present = [m in source_plate_df.index for m in mixing_table_df.columns]

In [4]:
present

[True, True, True, True, True, True, True, True, True, True, True, False]

i've been trying desperately and failing to write code to automatically update volumes in source plate but this doesn't work and my brain is tired

In [281]:
file_name = "C:/Users/rycar/Documents/GitHub/WHISPR/source_plates/220107_blCRISPRa.xlsx"
source_plate_df = pd.read_excel(file_name, sheet_name = 'ECHO_source_plate', index_col = 0)
vol_dict = {'A1': 20.1, 'C7': 1.35, 'C15': 2.2, 'C17': 1.2, 'B4': 4.5, 'B1': 2.475, 
'B2': 0.225, 'D1': 0.125, 'D8': 3.95, 'D12': 0.55, 'B14': 6.0, 'B15': 0.45, 'B16': 9.6, 
'B17': 0.8, 'E1': 0.6, 'E2': 0.1, 'F3': 0.6, 'F4': 0.05, 'F5': 0.05, 'F7': 0.075}

for well, vol in vol_dict.items():
    #component = [c[0] for c in source_plate_df.iterrows() if well in c[1]['Well']]
    component = source_plate_df.loc[source_plate_df['Well'] == well]
    if len(component) == 0:
        component = 'Water'
    else:
        component = component['Label'][0]

    if type(source_plate_df['Volume'][component]) == str:
        vol_list = source_plate_df['Volume'][component].split(',')
        well_list = [c[1]['Well'] for c in source_plate_df.iterrows() if well in c[1]['Well']]
        well_list = well_list[0].split(',')
        well_idx = well_list.index(well)
        vol_list[well_idx] = float(vol_list[well_idx]) - vol
        vol_list = [str(v) for v in vol_list]
        vol_list = ','.join(vol_list)
        source_plate_df.loc[component, 'Volume'] = vol_list

    elif type(source_plate_df[source_plate_df['Well'] == well]['Volume'][0]) == float:
        source_plate_df.loc[source_plate_df['Label'] == component, 'Volume'] = source_plate_df[source_plate_df['Well'] == well]['Volume'] - vol

    
    



KeyError: 'TJ1_RFP_100x'

In [280]:
source_plate_df[source_plate_df['Well'] == well]['Volume'][0]

45.5